# Project Generate random quizzes

In [161]:
!wget https://raw.githubusercontent.com/jasperdebie/VisInfo/master/us-state-capitals.csv &> /dev/null
!sed -i 's/<br>//' us-state-capitals.csv # Clean extra <br> tags left in csv file
import csv
import random

with open('us-state-capitals.csv') as file:
  reader = csv.reader(file)
  next(reader)
  answer_key = {state: capital for state, capital, *_ in reader}


In [162]:
def generate_quizzes(answer_key, n):
  for i in range(1, n+1):
    states = list(answer_key.keys())
    random.shuffle(states)
    with open(f'capital_quiz_{i:02}', 'w') as quiz_file, \
         open(f'capital_answer_key_{i:02}', 'w') as answer_file:
      quiz_file.write(f'State Capital Quiz  -- Key: {i}\n\n')
      quiz_file.write(f'Name: _____________ Date: ________\n\n')
      answer_file.write(f'State Capital Quiz (Answer Key) -- Key: {i}\n\n')
      for question_number, state in enumerate(states, start=1):
        question, answer = generate_question_and_answer(state, answer_key, question_number)
        quiz_file.write(question)
        answer_file.write(f'{question_number:02}) {answer}\n')



def generate_question_and_answer(state, answer_key, question_number):
  capital = answer_key[state]
  options = list(random.sample(set(answer_key.values()) - {capital}, 3))
  answer_index = random.randint(0,3)
  options.insert(answer_index, capital)
  options = list(zip('ABCD', options))
  question = f'{question_number:02}) What is the capital of {state}?\n'
  answers = '\n'.join(f'({letter}) {city}' for letter, city in options)

  return question + answers + '\n\n', 'ABCD'[answer_index]


In [163]:
generate_quizzes(answer_key, 35)

In [164]:
!cat capital_answer_key_01 | head

State Capital Quiz (Answer Key) -- Key: 1

01) D
02) B
03) C
04) A
05) B
06) C
07) B
08) B


In [165]:
!cat capital_quiz_01 | head

State Capital Quiz  -- Key: 1

Name: _____________ Date: ________

01) What is the capital of South Dakota?
(A) Madison
(B) Boise
(C) Bismarck
(D) Pierre



# Project: Updatable Multi-Clipboard

In [166]:
%%writefile mcp.py
#!/usr/bin/env python
"""Save/load text values in a local database using the command line."""
import argparse
import shelve

SHELF = 'local_db'


def main():
  """Save or load text from the db."""
  args = get_args()
  command = args.command
  with shelve.open(SHELF) as db:
    if command == 'save':
      db[args.key] = args.value
    elif command == 'load':
      print(db.get(args.key, f'Could not find key: {args.key!r}'))
    else:
      print('Keys:', ', '.join(db.keys()))


def get_args():
  """Get arguments from command line."""
  parser = argparse.ArgumentParser(
      description="Load or save strings from a local db file")
  parser.add_argument('command',
                      choices=('save', 'load', 'list'),
                      help='Save/load a value or list all keys')
  command = parser.parse_known_args()[0].command
  if command != 'list':
    parser.add_argument('key', help='Key to save to or load from', default='')
    if command == 'save':
      parser.add_argument('value', help='Value to save to key', default='')
  return parser.parse_args()


if __name__ == '__main__':
  main()


Writing mcp.py


In [167]:
!python mcp.py save test 'hello world'

In [168]:
!python mcp.py save world 'save the world!'

In [169]:
!python mcp.py list

Keys: test, world


In [170]:
!python mcp.py load world

save the world!


In [171]:
!python mcp.py load test

hello world


In [172]:
!python mcp.py load fake_key

Could not find key: 'fake_key'


In [173]:
!python mcp.py save test 'new test'

In [174]:
!python mcp.py load test

new test


# Practice Projects

## Extend the multiclipboard

Add delete command

In [175]:
%%writefile mcp.py
#!/usr/bin/env python
"""Save/load text values in a local database using the command line."""
import argparse
import shelve

SHELF = 'local_db'


def main():
  """Save or load text from the db."""
  args = get_args()
  command = args.command
  with shelve.open(SHELF) as db:
    if command == 'save':
      db[args.key] = args.value
    elif command == 'load':
      print(db.get(args.key, f'Could not find key: {args.key!r}'))
    elif command =='delete':
      try:
        del db[args.key]
      except KeyError:
        print(f'Could not find key: {args.key!r}')
    else:
      print('Keys:', ', '.join(db.keys()))


def get_args():
  """Get arguments from command line."""
  parser = argparse.ArgumentParser(
      description="Load or save strings from a local db file")
  parser.add_argument('command',
                      choices=('save', 'load', 'list', 'delete'),
                      help='Save/load a value or list all keys')
  command = parser.parse_known_args()[0].command
  if command != 'list':
    parser.add_argument('key', help='Key to save to or load from', default='')
    if command == 'save':
      parser.add_argument('value', help='Value to save to key', default='')
  return parser.parse_args()


if __name__ == '__main__':
  main()


Overwriting mcp.py


In [176]:
!python mcp.py delete test

In [177]:
!python mcp.py load test

Could not find key: 'test'


In [178]:
!python mcp.py delete fake_name

Could not find key: 'fake_name'


## Mad Libs

In [191]:
%%writefile madlibs.py
#!/usr/bin/env python
"""Ask user for words and print a madlib of them."""

TEMPLATE = 'The {} panda walked to the {} and then {}. A nearby {} was ' \
           'unaffected by these events.'
WORDS = ['an adjective', 'a noun', 'a verb', 'a noun']

def main():
  """Print the template using the user input."""
  print(TEMPLATE.format(*(input(f'Enter {word}:\n') for word in WORDS)))


if __name__ == '__main__':
  main()


Overwriting madlibs.py


In [192]:
!python madlibs.py

Enter an adjective:
exhausted
Enter a noun:
statue of a mermaid
Enter a verb:
ate lunch
Enter a noun:
thief
The exhausted panda walked to the statue of a mermaid and then ate lunch. A nearby thief was unaffected by these events.


## Regex search


In [195]:
%%writefile file_a.txt
"Hello World" -some computer

234-324-1524
554-243-6635

cat@gmail.com

Writing file_a.txt


In [196]:
%%writefile file_b.txt
And then the officer (her phone number was 532-452-5634) declared "It is raining"

The officer even had an email address for her dog dog@gmail.paw

Writing file_b.txt


In [236]:
%%writefile text_search.py
#!/usr/bin/env python
"""Search the local directory for all .txt files using the given regular
expression"""
import argparse
import os
import re


def main():
  """Get args, search the local directory, and print results that match the
  regular expression."""
  expression = get_args().expression
  text_file_names = [file for file in os.listdir() if file.endswith('.txt')]
  results = []
  for file_name in text_file_names:
    with open(file_name) as file:
      matches = re.findall(expression, file.read())
      if matches:
        results += matches
  print('Results:')
  print('\n'.join(results))


def get_args():
  """Get args from command line."""
  parser = argparse.ArgumentParser(
      description='Search a directory\'s txt files for a regular expression')
  parser.add_argument('expression', help='regular expression to search for')
  return parser.parse_args()


if __name__ == '__main__':
  main()


Overwriting text_search.py


In [237]:
!chmod +x text_search.py

In [238]:
!./text_search.py '".*"'

Results:
"It is raining"
"Hello World"


In [239]:
!./text_search.py '\d\d\d-\d\d\d-\d\d\d\d'

Results:
532-452-5634
234-324-1524
554-243-6635


In [241]:
!./text_search.py '\S*@\S*.\S*'

Results:
dog@gmail.paw
cat@gmail.com
